# MesoWest Data

# External Dataset

## [MesoWest](https://mesowest.utah.edu/)
In the example below the MesoWest surface observations will be used to make a time series plot for comparison with WRF.  The surface temperature and wind speed at one location will be plotted.  MesoWest has an API with a Python module to load data directly into Python.  You will need to register for an account.

* Register for a Token [here](https://developers.synopticdata.com/about/tokens/).

* Get the MesoPy module, do this from the Linux command line by:

> `python -m pip install MesoPy`

* Or in Python or a Notebook by typing:

> `import sys`

> `!{sys.executable} -m pip install MesoPy`

In the code cell below there is an example using this module to get MesoWest data for the WRF test case.

## <font color=red> Import MesoWest Data </font>

# The example below extracts MesoWest data for one station ID

In [ ]:
import sys
!{sys.executable} -m pip install MesoPy

from MesoPy import Meso
import numpy as np
import datetime as DT

m = Meso(token='YOUR-TOKEN-HERE')

# Returns a time series from SLC airport station KSLC from Jan 27 00z to Jan 31 00z 2017
slctime = m.timeseries(stid='kslc', start='201701270000', end='201701310000')

lat = float(slctime["STATION"][0]["LATITUDE"])
lon = float(slctime["STATION"][0]["LONGITUDE"])
T = slctime["STATION"][0]["OBSERVATIONS"]['air_temp_set_1']
WS = slctime["STATION"][0]["OBSERVATIONS"]['wind_speed_set_1']
WD = slctime["STATION"][0]["OBSERVATIONS"]['wind_direction_set_1']
Tdp = slctime["STATION"][0]["OBSERVATIONS"]['dew_point_temperature_set_1']
time = slctime["STATION"][0]["OBSERVATIONS"]['date_time']

# Convert the time to a numpy array for plotting with WRF outputs later
MWtime = np.array(time,dtype='datetime64')
MWtime2 = np.datetime_as_string(MWtime,unit='m')

# Convert the Temperautre to Kelvin for plotting with WRF outputs later 
Tkelvin = np.array(T)+273.15     # Convert to K

import matplotlib.pyplot as plt

#Quick plot to make sure the data was read in and seems reasonable
plt.plot(time,T,label="Temp")
plt.plot(time,WS,label="Wind Speed")
plt.plot(time,Tdp,label="Dew Point")
plt.legend(loc="best", fontsize=14)

plt.show()

# The example below is to use a radius to get all stations

In [ ]:
# Collects sensor towers within 25 miles of downtown Chicago

radius = 25 # miles
latitude = 41.888636
longitude = -87.631688
cook = m.metadata(state = 'IL', county = 'Cook', radius = (latitude,longitude,radius))
dupage = m.metadata(state = 'IL', county = 'Dupage', radius = (latitude,longitude,radius))
will = m.metadata(state = 'IL', county = 'Will', radius = (latitude,longitude,radius)) 

### The code below uses the results from the code block above, it removes stations with NaNs based on time

In [ ]:
#Removes sensor towers that were not active in 2012
def checktime(county):
    county_towers = county["STATION"]
    county_STID = []
    
    for i in range(len(county_towers)):
        try:
            start_year = int(county_towers[i]["PERIOD_OF_RECORD"]["start"][0:4])
            end_year = int(county_towers[i]["PERIOD_OF_RECORD"]["end"][0:4])

            if start_year < 2012 and end_year > 2012: # makes sure the station was active in 2012
                county_STID.append(county_towers[i]["STID"])

        except: # some of the stations don't have start,end so we won't add them
            ''
            
    return(county_STID)
    
STID = checktime(cook) + checktime(dupage) + checktime(will)  